In [ ]:
from __future__ import print_function, division
from os.path import dirname, realpath, join
import os
import sys

import numpy as np
import cv2
import yaml
import pandas as pd

import torchvision.transforms as transforms
import matplotlib.pyplot as plt

from PIL import ImageDraw

In [ ]:
class Canvas:
    def __init__(self, draw):
        self.draw = draw
    
    def DrawLine(self, point1, point2, lineColor, lineWidth):
        '''Draws line on image'''
        if not point1 is None and point2 is not None:
            self.draw.line([point1,point2], fill=lineColor, width=lineWidth)

    def DrawDot(self, point, pointColor, pointRadius):
        '''Draws dot (filled circle) on image'''
        if point is not None:
            xy = [
                point[0]-pointRadius, 
                point[1]-pointRadius, 
                point[0]+pointRadius, 
                point[1]+pointRadius
            ]
            self.draw.ellipse(xy, 
                fill=pointColor, 
                outline=pointColor
            )

    def DrawCube(self, points, color=(255, 0, 0)):
        '''
        Draws cube with a thick solid line across 
        the front top edge and an X on the top face.
        '''

        lineWidthForDrawing = 2
        
        points = [tuple(pt) for pt in points]

        # draw front
        self.DrawLine(points[0], points[1], color, lineWidthForDrawing)
        self.DrawLine(points[1], points[2], color, lineWidthForDrawing)
        self.DrawLine(points[3], points[2], color, lineWidthForDrawing)
        self.DrawLine(points[3], points[0], color, lineWidthForDrawing)

        # draw back
        self.DrawLine(points[4], points[5], color, lineWidthForDrawing)
        self.DrawLine(points[6], points[5], color, lineWidthForDrawing)
        self.DrawLine(points[6], points[7], color, lineWidthForDrawing)
        self.DrawLine(points[4], points[7], color, lineWidthForDrawing)

        # draw sides
        self.DrawLine(points[0], points[4], color, lineWidthForDrawing)
        self.DrawLine(points[7], points[3], color, lineWidthForDrawing)
        self.DrawLine(points[5], points[1], color, lineWidthForDrawing)
        self.DrawLine(points[2], points[6], color, lineWidthForDrawing)

        # draw dots
        self.DrawDot(points[0], pointColor=color, pointRadius = 4)
        self.DrawDot(points[1], pointColor=color, pointRadius = 4)

        # draw x on the top 
        self.DrawLine(points[0], points[5], color, lineWidthForDrawing)
        self.DrawLine(points[1], points[4], color, lineWidthForDrawing)

In [ ]:
cur = os.getcwd()

In [ ]:
sys.path.append(join(cur, "training"))
sys.path.append(join(cur))

In [ ]:
from train import MultipleVertexJson
from validate import Comparison, Pose, load_model_cm, get_mesh_file, add_metric, get_dataset
from cuboid import Cuboid3d

In [ ]:
model = "003_cracker_box"
model_cm = load_model_cm(get_mesh_file(model))
def pose_error(X_est, X_gt):
    return add_metric(X_est.R, X_est.t_cm, X_gt.R, X_gt.t_cm, model_cm)

In [ ]:
import cPickle as pickle

In [ ]:
with open("../comp_list.pkl") as f:
    comp_list = pickle.load(f)

In [ ]:
from tqdm import tqdm_notebook

In [ ]:
def compute(error_threshold, skip_no_gt=True):
    metrics = []
    for i, comp in enumerate(comp_list):
        if comp.num_gt == 0 and skip_no_gt:
            continue
        metrics.append(comp.compute_metrics(pose_error, error_threshold))
    metrics = np.hstack(metrics)
    return pd.DataFrame(metrics).mean()

In [ ]:
ets = []
metrics = []
for error_threshold in tqdm_notebook(np.linspace(0, 10, 100)):
    ets.append(error_threshold / 100)
    metrics.append(compute(error_threshold))
metrics = pd.DataFrame(metrics)

In [ ]:
plt.plot(ets, metrics["precision"])
plt.xlabel("ADD Metric (m)")
plt.ylabel("F1 Score")
plt.title(model)
plt.xlim([0, 0.1])
plt.ylim([0, 1])
plt.savefig("/home/eacousineau/proj/tri/proj/perception/plots/simple_stuff.png")

In [ ]:
dataset = get_dataset(model)

In [ ]:
def get_cube(v):
    v_min = np.min(v, axis=0)
    v_max = np.max(v, axis=0)
    v_ex = v_max - v_min
    v_c = (v_max + v_min) / 2
    return Cuboid3d(size3d=v_ex, center_location=v_c)

def project_2d(K, X_CO, v_O_cm):
    assert v_O_cm.shape[0] == 3
    # Meh. Too lazy to figure out how to get rvec for cv2.
    v_C_cm = np.dot(X_CO.R, v_O_cm) + X_CO.t_cm.reshape((-1, 1))
    v_C_n = v_C_cm / v_C_cm[[2], :]
    return np.dot(K, v_C_n)[:2]

def cube_project_2d(cube, K, X_CO):
    v = np.array(cube.get_vertices())
    return project_2d(K, X_CO, v.T).T

In [ ]:
model_cube = get_cube(model_cm["pts"])

In [ ]:
for i, comp in enumerate(comp_list):
    if comp.num_gt > 0 and comp.num_est > 0:
        break
else:
    assert False
print(i)

In [ ]:
target = dataset[i]
X_CO_est = comp.pose_est_list[0]
X_CO_gt = comp.pose_gt_list[0]

K = target["matrix_camera"]
img = target["img"]
c = Canvas(ImageDraw.Draw(img))

In [ ]:
# points_gt = target["pointsBelief"]
points_gt = cube_project_2d(model_cube, K, X_CO_gt)
points_est = cube_project_2d(model_cube, K, X_CO_est)

In [ ]:
c.DrawCube(points_gt, (0, 255, 0))
c.DrawCube(points_est, (255, 0, 0))
#c.DrawLine(points[0], points[1], (255, 0, 0), 2)

In [ ]:
img